In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme()

In [2]:
base_dir = '/Users/das/work/svn/Gits/a_Projects/101_clembench/clembench-runs'

modprop_df = pd.read_csv(base_dir + '/Addenda/ModelAnalysis2024/Results/model_characteristics.csv')

In [3]:
### TEMPORARY, until PR is approved and file is at correct locaiton
modprop_df = pd.read_csv('In/TMP_model_characteristics.csv')

In [4]:
modprop_df['Release Date'] = pd.to_datetime(modprop_df['Release Date'])
modprop_df['Cut-off Date'] = pd.to_datetime(modprop_df['Cut-off Date'])

## well fuck me. Turns out that the model names can contain trailing whitespaces...
modprop_df['Model Name'] = modprop_df['Model Name'].str.strip()


v16 = pd.read_csv(base_dir + '/v1.6/results.csv')
#v16.loc[:,'model'] = v16.iloc[:,0].str.replace(r'(\w*)-(?:hf-)?t0.0.*', r'\1').str.lower()
v16.loc[:,'model'] = v16.iloc[:,0].str.replace(r'(\w*)-t0.0.*', r'\1').str.lower()


## f me part II: turns out that the results.csv has openchat_3.5, and modprop has openchat-3.5
##  normalise to the latter
v16.loc[v16.model == 'openchat_3.5', 'model'] = 'openchat-3.5'

merged = pd.merge(left=modprop_df, right=v16, left_on='Model Name', right_on='model')

(
 merged.model,
 set(v16.model) - set(merged.model)
)

/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_74904/4263828293.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  v16.loc[:,'model'] = v16.iloc[:,0].str.replace(r'(\w*)-t0.0.*', r'\1').str.lower()


(0                        yi-34b-chat
 1                     yi-1.5-6b-chat
 2                     yi-1.5-9b-chat
 3                    yi-1.5-34b-chat
 4                     tulu-2-dpo-70b
 5                         claude-2.1
 6            claude-3-haiku-20240307
 7           claude-3-sonnet-20240229
 8             claude-3-opus-20240229
 9           dolphin-2.5-mixtral-8x7b
 10                         command-r
 11                    command-r-plus
 12                   gemma-1.1-7b-it
 13                       gemma-7b-it
 14           gemini-1.5-flash-latest
 15             gemini-1.5-pro-latest
 16                    gemini-1.0-pro
 17                   codegemma-7b-it
 18                   gemma-1.1-2b-it
 19       meta-llama-3-8b-instruct-hf
 20      meta-llama-3-70b-instruct-hf
 21               llama-2-70b-chat-hf
 22         codellama-34b-instruct-hf
 23          mistral-7b-instruct-v0.1
 24        mixtral-8x7b-instruct-v0.1
 25       mixtral-8x22b-instruct-v0.1
 26         

In [5]:
merged.columns = merged.columns.str.strip()

In [6]:
merged.columns

Index(['Model Provider', 'Model Name', 'Training Data Size (in trillions)',
       'Cut-off Date', 'Parameter Size (in billions)', 'Release Date',
       'Context Length (in thousands)', 'Commercial', 'Multilingual',
       'Multimodal', 'Base Model', 'Instruction Tuning Data', 'Unnamed: 0',
       '-, clemscore', 'all, Average % Played', 'all, Average Quality Score',
       'imagegame, % Played', 'imagegame, Quality Score',
       'imagegame, Quality Score (std)', 'privateshared, % Played',
       'privateshared, Quality Score', 'privateshared, Quality Score (std)',
       'referencegame, % Played', 'referencegame, Quality Score',
       'referencegame, Quality Score (std)', 'taboo, % Played',
       'taboo, Quality Score', 'taboo, Quality Score (std)',
       'wordle, % Played', 'wordle, Quality Score',
       'wordle, Quality Score (std)', 'wordle_withclue, % Played',
       'wordle_withclue, Quality Score',
       'wordle_withclue, Quality Score (std)', 'wordle_withcritic, % Played

In [7]:
(
    merged[merged['Base Model']
           .isin([b for b,c in merged['Base Model'].value_counts().items() if c > 1])]
    [['Model Provider', 'Model Name', 'Release Date', 'Base Model',
      '-, clemscore',  'all, Average % Played', 'all, Average Quality Score',
      'Instruction Tuning Data']]
    .sort_values(by=['Base Model', '-, clemscore'])
)

,Model Provider,Model Name,Release Date,Base Model,"-, clemscore","all, Average % Played","all, Average Quality Score",Instruction Tuning Data
13,Google,gemma-7b-it,2024-02-01,gemma-7b,1.82,17.78,10.23,NaN
12,Google,gemma-1.1-7b-it,2024-04-01,gemma-7b,14.14,49.67,28.46,NaN
17,Google,codegemma-7b-it,2024-04-01,gemma-7b,15.30,51.95,29.45,NaN
21,Meta,llama-2-70b-chat-hf,2023-07-01,llama-2-70b-hf,0.81,7.14,11.31,NaN
4,Allenai,tulu-2-dpo-70b,2024-11-01,llama-2-70b-hf,12.62,49.76,25.37,"ultrafeedback-binarized, tulu-v2-sft-mixture"
47,Wizardlmteam,wizardlm-70b-v1.0,2023-08-01,llama-2-70b-hf,17.40,46.19,37.66,wizardlm-evol-instruct-v2-196k
46,Riid,sheep-duck-llama-2-70b-v1.1,2023-09-01,llama-2-70b-hf,21.50,41.19,52.20,orca and alpaca inspired data
29,Nexusflow,starling-lm-7b-beta,2024-03-01,mistral-7b-v0.1,6.56,30.89,21.25,nectar
23,Mistralai,mistral-7b-instruct-v0.1,2023-09-01,mistral-7b-v0.1,8.01,37.14,21.58,NaN
39,Openchat,openchat-3.5-0106,2024-01-01,mistral-7b-v0.1,17.10,52.57,32.52,"sharegpt, openorca, capybara, goat, glaive, me..."


In [8]:
with pd.option_context("max_colwidth", 1000):
    (
        merged[merged['Base Model']
               .isin([b for b,c in merged['Base Model'].value_counts().items() if c > 1])]
        [['Base Model', 'Model Name', 'Model Provider', 'Release Date', 
          '-, clemscore',  'all, Average % Played', 'all, Average Quality Score',
          'Instruction Tuning Data']]
        .sort_values(by=['Base Model', '-, clemscore'])
        .to_latex('Out/same_base_results.tex',
                  header=['Base Model', 'Model Name', 'Model Provider', 'Release Date', 
                          'cs',  '% pl', 'q sc',
                          'Instruction Tuning Data'],
                  column_format='TTllrrrp{.3\linewidth}',
                  index=False,
                  float_format="%.2f")
    )

/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_74904/3948124813.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  merged[merged['Base Model']


In [9]:
modprop_df.loc[modprop_df['Model Name'] == 'mixtral-8x7b-instruct-v0.1']

,Model Provider,Model Name,Training Data Size (in trillions),Cut-off Date,Parameter Size (in billions),Release Date,Context Length (in thousands),Commercial,Multilingual,Multimodal,Base Model,Instruction Tuning Data
27,Mistralai,mixtral-8x7b-instruct-v0.1,NaN,NaT,45.0,2023-12-01,32,0,1,0,mixtral-8x7b-v0.1,NaN


In [10]:
modprop_df.loc[modprop_df['Model Name'] == 'openchat-3.5']

,Model Provider,Model Name,Training Data Size (in trillions),Cut-off Date,Parameter Size (in billions),Release Date,Context Length (in thousands),Commercial,Multilingual,Multimodal,Base Model,Instruction Tuning Data
40,Openchat,openchat-3.5,NaN,NaT,7.0,2023-10-01,8,0,0,0,mistral-7b-v0.1,"sharegpt, openorca, capybara, goat, glaive, me..."
